## Installation

Installing packages needed to run DNABERT2


In [ ]:
!pip install transformers==4.29.2 scikit-learn einops evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


Uninstall triton in shell; run under T4 with High-RAM

In [ ]:
!pip uninstall --yes triton

Found existing installation: triton 2.1.0
Uninstalling triton-2.1.0:
  Successfully uninstalled triton-2.1.0


Access TFBS dataset in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set kmer and model parameters

In [ ]:
kmer = 6
#model_used = "armheb/DNA_bert_" + str(kmer)
model_used = "zhihan1996/DNABERT-2-117M"
train_bs = 32
eval_bs = 32
epochs = 3
warmup = 75
lr = 3e-5
save_steps = 200
eval_steps = 200
save_total_limit = 3

run_name = "run-" + "gue-test-db2-git-62"

Set output path

In [ ]:
# set output path

path_prefix = "/content/drive/MyDrive/"
# stdpath = path_prefix + "DNABERT/output/" + "optimization" + "/"

Run model on next datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import sys
import os

# set up dataframe to capture testing results

test_column_names = ['test_loss', 'test_accuracy', 'test_precision', 'test_recall',
                     'test_f1', 'test_matthews_correlation', 'test_runtime']
testing_results_df = pd.DataFrame(columns=test_column_names)

# initialize parameters

for fname in [
#  "UtaHuvecPol2UniPk601-ran.csv",
#  "UtaProgfibPol2UniPk601-ran.csv",
#  "HaibGm12892Pax5c20V0416101UniPk601-ran.csv",
  "SydhU2osKap1UcdUniPk601-ran.csv",
  "SydhH1hescMxi1IggrabUniPk601-ran.csv"
#  "BroadDnd41Ezh239875UniPk-ran.csv",
#  "BroadGm12878CtcfUniPk-ran.csv",
#  "BroadGm12878Ezh239875UniPk-ran.csv",
#  "BroadH1hescChd1a301218aUniPk-ran.csv",
#  "BroadH1hescCtcfUniPk-ran.csv",
#  "BroadH1hescEzh239875UniPk-ran.csv",
#  "BroadH1hescJarid1aab26049UniPk-ran.csv",
#  "BroadH1hescRbbp5a300109aUniPk-ran.csv",
#  "BroadHelas3CtcfUniPk-ran.csv",
#  "BroadHelas3Ezh239875UniPk-ran.csv",
#  "BroadHelas3Pol2bUniPk-ran.csv",
#  "BroadHepg2CtcfUniPk-ran.csv",
#  "BroadHepg2Ezh239875UniPk-ran.csv",
#  "BroadHmecCtcfUniPk-ran.csv",
#  "BroadHmecEzh239875UniPk-ran.csv",
#  "BroadHsmmCtcfUniPk-ran.csv",
#  "BroadHsmmEzh239875UniPk-ran.csv",
#  "BroadHsmmtCtcfUniPk-ran.csv",
#  "BroadHsmmtEzh239875UniPk-ran.csv",
#  "BroadHuvecCtcfUniPk-ran.csv",
#  "BroadHuvecEzh239875UniPk-ran.csv",
#  "BroadHuvecPol2bUniPki-ran.csv",
#  "BroadK562Chd1a301218aUniPk-ran.csv",
#  "BroadK562CtcfUniPk-ran.csv",
#  "BroadK562Ezh239875UniPk-ran.csv",
#  "BroadK562Hdac1sc6298UniPk-ran.csv",
#  "BroadK562Hdac2a300705aUniPk-ran.csv",
#  "BroadK562Hdac6a301341aUniPk-ran.csv",
#  "BroadK562P300UniPk-ran.csv",
#  "BroadK562Phf8a301772aUniPk-ran.csv",
#  "BroadK562Plu1UniPk-ran.csv",
#  "BroadK562Pol2bUniPk-ran.csv",
#  "BroadK562Rbbp5a300109aUniPk-ran.csv",
#  "BroadK562Sap3039731UniPk-ran.csv",
#  "BroadNhaCtcfUniPk-ran.csv",
#  "BroadNhaEzh239875UniPk-ran.csv",
#  "BroadNhdfadCtcfUniPk-ran.csv",
#  "BroadNhdfadEzh239875UniPk-ran.csv",
#  "BroadNhekCtcfUniPk-ran.csv",
#  "BroadNhekEzh239875UniPk-ran.csv",
#  "BroadNhekPol2bUniPk-ran.csv",
#  "BroadNhlfCtcfUniPk-ran.csv",
#  "BroadNhlfEzh239875UniPk-ran.csv",
#  "BroadOsteoblCtcfUniPk-ran.csv",
#  "HaibA549Atf3V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Bcl3V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Creb1sc240V0416102Dex100nmUniPk-ran.csv",
#  "HaibA549Ctcfsc5916Pcr1xDex100nmUniPk-ran.csv",
#  "HaibA549Ctcfsc5916Pcr1xEtoh02UniPk-ran.csv",
#  "HaibA549Elf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Ets1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Fosl2V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Foxa1V0416102Dex100nmUniPk-ran.csv",
#  "HaibA549GabpV0422111Etoh02UniPk-ran.csv",
#  "HaibA549GrPcr1xDex500pmUniPk-ran.csv",
#  "HaibA549GrPcr1xDex50nmUniPk-ran.csv",
#  "HaibA549GrPcr1xDex5nmUniPk-ran.csv",
#  "HaibA549GrPcr2xDex100nmUniPk-ran.csv",
#  "HaibA549NrsfV0422111Etoh02UniPk-ran.csv",
#  "HaibA549P300V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Pol2Pcr2xDex100nmUniPk-ran.csv",
#  "HaibA549Pol2Pcr2xEtoh02UniPk-ran.csv",
#  "HaibA549Sin3ak20V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Six5V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Taf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Tcf12V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Usf1Pcr1xDex100nmUniPk-ran.csv",
#  "HaibA549Usf1Pcr1xEtoh02UniPk-ran.csv",
#  "HaibA549Usf1V0422111Etoh02UniPk-ran.csv",
#  "HaibA549Yy1cV0422111Etoh02UniPk-ran.csv",
#  "HaibA549Zbtb33V0422111Etoh02UniPk-ran.csv",
#  "HaibEcc1CtcfcV0416102Dm002p1hUniPk-ran.csv",
#  "HaibEcc1EraaV0416102Bpa1hUniPk-ran.csv",
#  "HaibEcc1EralphaaV0416102Est10nm1hUniPk-ran.csv",
#  "HaibEcc1EralphaaV0416102Gen1hUniPk-ran.csv",
#  "HaibEcc1Foxa1sc6553V0416102Dm002p1hUniPk-ran.csv",
#  "HaibEcc1GrV0416102Dex100nmUniPk-ran.csv",
#  "HaibEcc1Pol2V0416102Dm002p1hUniPk-ran.csv",
]:

  dsname = path_prefix + "DNABERT/Datasets/tfbs/" + fname
  tfbs_dataset = pd.read_csv(dsname, sep=',')

# change standard error and output to saved output file

  stdpath = path_prefix + "DNABERT/output/" + fname.split(".")[0] + "/"

  try:
    os.makedirs(stdpath)
  except FileExistsError:
    pass

  stdpath_out = stdpath + "stdout.txt"
  stdpath_err = stdpath + "stderr.txt"

  sys.stdout = open(stdpath_out, 'w')
  sys.stderr = open(stdpath_err, 'w')

# print dataset shape and initial values

  print("tfbs dataset:", tfbs_dataset.shape, file=sys.stdout)
  print("tfbs dataset initial values:", tfbs_dataset.head(), file=sys.stdout)

# reformat input

  column_names = ["labels", "seq"]
  tfbs_dataset_res = pd.DataFrame(columns=column_names)

  j = 1
  for i in range(tfbs_dataset.count()['names']):
    name = tfbs_dataset['names'][i]
    pos_seq = tfbs_dataset['seq'][i]
    neg_seq = tfbs_dataset['neg_seq'][i]
    pos_label = 1
    neg_label = 0
    tfbs_dataset_res.loc[j] = [pos_label, pos_seq]
    tfbs_dataset_res.loc[j+1] = [neg_label, neg_seq]
    j+=2

  X = tfbs_dataset_res['seq']
  y = tfbs_dataset_res['labels']
  X_size = len(X)

# test range is 500 odd-numbered pairs of sequences
# training range is 500 even-numbered pairs plus remainder of the sequences over 1000

  test_range = list(range(3,2001,4)) + list(range(4,2001,4))
  test_range.sort()
  train_range = list(range(1,2001,4)) + list(range(2,2001,4))
  train_range.sort()
  train_range = train_range + list(range(2001,X_size))

  X_test = X.loc[test_range]
  X_train = X.loc[train_range]
  y_test = y.loc[test_range]
  y_train = y.loc[train_range]


# print size of training and test sets

  print("X_train shape:", X_train.shape, file=sys.stdout)
  print("y_train shape:", y_train.shape, file=sys.stdout)
  print("X_test shape:", X_test.shape, file=sys.stdout)
  print("y_test shape:", y_test.shape, file=sys.stdout)


# load pre-trained model

  model_cls = AutoModelForSequenceClassification.from_pretrained(model_used, num_labels=2, trust_remote_code=True)
  params = list(model_cls.named_parameters())
  tokenizer = AutoTokenizer.from_pretrained(model_used, model_max_length=30, padding_side="right", use_fast=True, trust_remote_code=True)

# reformat data to Hugging Face Dataset format from pandas

  ds_Xy_train = pd.concat([y_train, X_train], axis=1)
  ds_Xy_test = pd.concat([y_test, X_test], axis=1)

  Dataset_Xy_train = Dataset.from_pandas(ds_Xy_train)
  Dataset_Xy_test = Dataset.from_pandas(ds_Xy_test)
  Dataset_Xy_train, Dataset_Xy_test

  def tokenize(batch):
    return tokenizer(batch["seq"], return_tensors="pt", padding='longest', max_length=30, truncation=True)

  Dataset_Xy_train_tok = Dataset_Xy_train.map(tokenize, batched=True, batch_size=None)
  new_column = ["train"] * len(Dataset_Xy_train_tok)
  Dataset_Xy_train_tok = Dataset_Xy_train_tok.add_column("dset", new_column)

  Dataset_Xy_test_tok = Dataset_Xy_test.map(tokenize, batched=True, batch_size=None)
  new_column = ["test"] * len(Dataset_Xy_test_tok)
  Dataset_Xy_test_tok = Dataset_Xy_test_tok.add_column("dset", new_column)

  dds = DatasetDict({
    'train': Dataset_Xy_train_tok,
    'test': Dataset_Xy_test_tok
  })

# switch to GPU

  model_cls = model_cls.to('cuda')

# train model

  output_dir = path_prefix + 'outputs'

  args = TrainingArguments(output_dir, learning_rate=lr, warmup_steps=warmup, fp16=True,
    evaluation_strategy="steps", per_device_train_batch_size=train_bs, per_device_eval_batch_size=eval_bs,
    eval_steps=eval_steps, save_steps=save_steps, logging_steps=100000, save_total_limit=save_total_limit,
    load_best_model_at_end=True, num_train_epochs=epochs, weight_decay=0.01, report_to='none')

  def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy","precision","recall","f1","matthews_correlation"])
    logits, labels = eval_preds
    if isinstance(logits, tuple):  # Unpack logits if it's a tuple
      logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  trainer.train()

# save model

  fpath = path_prefix + "DNABERT/Output_Models/" + fname.split(".")[0] + "/"
  print(fpath, file=sys.stdout)
  trainer.save_model(fpath)

# test model

  eval_preds = trainer.predict(dds['test'])

  print(eval_preds, file=sys.stdout)

  tfbs_ds = fname.split(".")[0]
  testing_results_df.loc[tfbs_ds] = [eval_preds.metrics['test_loss'],
                                  eval_preds.metrics['test_accuracy'],
                                  eval_preds.metrics['test_precision'],
                                  eval_preds.metrics['test_recall'],
                                  eval_preds.metrics['test_f1'],
                                  eval_preds.metrics['test_matthews_correlation'],
                                  eval_preds.metrics['test_runtime']]

# save results in output file

outpath = path_prefix + "DNABERT/output/" + run_name + ".csv"
testing_results_df.to_csv(outpath)


(…)/DNABERT-2-117M/resolve/main/config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

(…)-117M/resolve/main/configuration_bert.py:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)ABERT-2-117M/resolve/main/bert_layers.py:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

(…)BERT-2-117M/resolve/main/bert_padding.py:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly ini

Map:   0%|          | 0/11701 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Matthews Correlation
200,No log,0.582615,0.700000,0.801205,0.532000,0.639423,0.424691
400,No log,0.524936,0.753000,0.722320,0.822000,0.768943,0.510888
600,No log,0.558439,0.731000,0.795396,0.622000,0.698092,0.473386
800,No log,0.533803,0.746000,0.743083,0.752000,0.747515,0.492035
1000,No log,0.551095,0.743000,0.768212,0.696000,0.730325,0.488161


In [ ]:
testing_results_df

,test_loss,test_accuracy,test_precision,test_recall,test_f1,test_matthews_correlation,test_runtime
SydhU2osKap1UcdUniPk601-ran,0.594684,0.673,0.65930,0.716,0.686481,0.347287,9.2685
SydhH1hescMxi1IggrabUniPk601-ran,0.524936,0.753,0.72232,0.822,0.768943,0.510888,9.2736
